In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

In [2]:
#can use data augmentation in this line of code

In [3]:
#read dataset, it combine all three folders into one
dataset = datasets.ImageFolder(root="F:/dataset/potato_disease", 
                               transform=ToTensor())

In [4]:
#we must split into 3 separate classes, 
#because if we just randomly split into train/dev/test, they might be very skewed
early_blight = []
healthy = []
late_blight = []
for ex in range(len(dataset)):
    if dataset[ex][1] == 0:
        early_blight.append(dataset[ex])
    elif dataset[ex][1] ==1: 
        healthy.append(dataset[ex])
    else:
        late_blight.append(dataset[ex])
len(early_blight), len(healthy), len(late_blight)

(1000, 152, 1000)

In [9]:
#custom datasets that are the same as above (list) but with extra Pytorch features
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        x, y = self.data[idx]
        #change label to tensor too
        y = torch.tensor(y, dtype=torch.long)
        return x, y

In [10]:
#change healthy, early_blight, late_blight to dataset, then split into train and test set, and then concat all three of them
## -> finally turn them into train_dataloader, test_dataloader
healthy_data = CustomDataset(healthy)
train_size = int(0.8*len(healthy_data))
test_size = len(healthy_data) - train_size
healthy_train, healthy_test = random_split(healthy_data, [train_size, test_size])

early_data = CustomDataset(early_blight)
train_size = int(0.8*len(early_data))
test_size = len(early_data) - train_size
early_train, early_test = random_split(early_data, [train_size, test_size])

late_data = CustomDataset(late_blight)
train_size = int(0.8*len(late_data))
test_size = len(late_data) - train_size
late_train, late_test = random_split(late_data, [train_size, test_size])

#concat them using ConcatDataset
train_data = torch.utils.data.ConcatDataset([healthy_train, early_train, late_train])
test_data = torch.utils.data.ConcatDataset([healthy_test, early_test, late_test])

#turn them into DataLoader
train_dataloader = DataLoader(train_data, batch_size = 32, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size = 32, shuffle=False)

In [11]:
for images, labels in train_dataloader:
    print(images.shape)   # e.g. [32, 3, 224, 224]
    print(labels.shape)   # e.g. [32]
    break 

torch.Size([32, 3, 256, 256])
torch.Size([32])


In [13]:
#define CNN architecture
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 10, kernel_size = 3, padding = 1)
        self.conv2 = nn.Conv2d(10, 10, kernel_size = 3)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(0.3)
        self.fc1 = nn.Linear(63*63*10, 128)
        self.fc2 = nn.Linear(128, 3)
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 63*63*10)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [14]:
#setup device agnostic code
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [15]:
#create an instance
model = CNN().to(device)

#setup metrics, loss, optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model.parameters(),
                             lr = 1e-1)